# Bulk Properties

`yaeos` is a library based on Equation-of-State calculations. An Equation of
State (EOS) is a mathematical model that describes the relationship between
thermodynamic properties of a system, typically pressure (P), volume (V), and
temperature (T). These equations are fundamental in physics, chemistry, and
engineering, especially in thermodynamics and fluid mechanics.

EOS models are used to predict phase behavior, thermodynamic properties, and
equilibrium conditions of gases, liquids, solids, and mixtures. They are widely
applied in areas such as chemical engineering, petroleum engineering, and
material science. From the ideal gas law, to cubic equations of state and SAFT,
to more complex models, EOS are essential tools for the study and design of
processes.

The classic way of defining an EOS is though a mathematical equation that
express the pressure as a function of the volume and temperature. For example,
the Van der Waals EOS is defined as:

$$
    P = \frac{RT}{v - b} - \frac{a}{v^2}
$$

Where $P$ is the pressure, $v$ is the molar volume, $T$ is the
temperature, $R$ is the gas constant, and $a$ and $b$ are parameters that
depend on the substance. For mixtures, the $a$ and $b$ parameters are
calculated from the pure components parameters using a mixing rule.

The modern way of defining an EOS is through a mathematical model that express
the residual Helmholtz free energy as a function of the mole number vector,
volume and temperature. This approach is more general and allows for a more
flexible definition of the EOS in the context of implementing it in code. All
thermodynamic properties can be calculated from the residual Helmholtz free
energy and its derivatives. `yaeos` is based on this approach.

$$
    A^{r}(\vec{n}, V, T) = -\int_{\infty}^{V} \left(P(\vec{n},V,T) - \frac{nRT}{V} \right) dV
$$

If you want to know how each bulk property is calculated from the residual
Helmholtz free energy, please refer to the [Fortran User Documentation](https://ipqa-research.github.io/yaeos/page/index.html)

## Example model
First, we are going the import the `yaeos` library and instantiate a
Peng-Robinson (1976) model with two components. Later in this tutorial you will
learn got to instantiate different models, but for now we will use this one as
an example.

**The important thing to understand is that all the EoS can evaluate
the thermodynamic properties explained in this section.**

In [1]:
import yaeos


# Model parameters
Tc = [320, 375]      # critical temperatures [K]
Pc = [30, 45]        # critical pressures [bar]
w = [0.0123, 0.045]  # acentric factors [-]

model = yaeos.PengRobinson76(Tc, Pc, w)

## Properties function of Volume and Temperature $\left(V, T \right)$
We can obtain different bulk properties from a $A^r(\vec{n}, V, T)$ model
specifying the number of moles, volume and temperature. But also, derivatives
could be obtained if asked for.

All not asked derivatives will be set as `None`. The derivatives are returned
as a Python dictionary with the keys `dv`, `dt`, and `dn`, etc. respectively.
The compositional derivatives are always returned as `Numpy` arrays.


The following properties can be calculated:

### Pressure

In [2]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.pressure(n, V, T) # Calculate pressure [bar]

56.000067835576885

You can obtain the next derivatives:

$$
\left(\frac{\partial P}{\partial V}\right)_{T, \vec{n}} \quad
\left(\frac{\partial P}{\partial T}\right)_{V, \vec{n}} \quad
\left(\frac{\partial P}{\partial n_i}\right)_{V, T, n_{j \neq i}}
$$

Ask for derivatives as follows:

In [3]:
# Asking for all derivatives
pressure, derivatives = model.pressure(n, V, T, dv=True, dt=True, dn=True)

# Displaying results
print("Pressure: ", pressure)
print("Derivatives: ", derivatives)

Pressure:  56.000067835576885
Derivatives:  {'dv': -647.5628198115064, 'dt': 2.900736382506016, 'dn': array([78.64160081, 50.87096316])}


### Logarithm of fugacity coefficients $ln \, \phi(\vec{n},V,T)$

In [4]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.lnphi_vt(n, V, T)

array([-1.12084851, -1.61086   ])

You can obtain the next derivatives:

$$
\left(\frac{\partial ln \, \hat{\phi}_i}{\partial T}\right)_{P, \vec{n}} \quad
\left(\frac{\partial ln \, \hat{\phi}_i}{\partial P}\right)_{T, \vec{n}} \quad
\left(\frac{\partial ln \, \hat{\phi}_i}{\partial n_j}\right)_{P, T, n_{k \neq j}}
$$


Ask for derivatives as follows:

In [7]:
# Asking for all the derivatives
lnphi, derivatives = model.lnphi_vt(n, V, T, dt=True, dp=True, dn=True)

# Displaying results
print("Logarithm of Fugacity Coefficients: ", lnphi)
print("Derivatives: ", derivatives)

Logarithm of Fugacity Coefficients:  [-1.12084851 -1.61086   ]
Derivatives:  {'dt': array([0.01380036, 0.01926589]), 'dp': array([-0.0129884 , -0.01470769]), 'dn': array([[-0.00904545,  0.00904545],
       [ 0.00904545, -0.00904545]])}


You can evaluate fugacity by simply:

In [8]:
import numpy as np


p = model.pressure(n, V, T)  # Calculate pressure [bar]

z = n / np.sum(n)            # Calculate mole fractions

fugacities = np.exp(lnphi) * z * p

print("Fugacities: ", fugacities)

Fugacities:  [9.12809679 5.59204872]


### Residual Gibbs Energy $G^r(\vec{n}, V, T)$

In [9]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.gibbs_residual_vt(n, V, T) # Calculate residual Gibbs energy [bar L]

-713.3048693974467

You can obtain the next derivatives:

$$
\left(\frac{\partial G^{r}_{(\vec{n}, V, T)}}{\partial T}\right)_{V, \vec{n}} \quad
\left(\frac{\partial G^{r}_{(\vec{n}, V, T)}}{\partial V}\right)_{T, \vec{n}} \quad
\left(\frac{\partial G^{r}_{(\vec{n}, V, T)}}{\partial n_j}\right)_{V, T, n_{i \neq j}}
$$

Ask for derivatives as follows:

In [11]:
# Asking for all the derivatives
Gr, derivatives = model.gibbs_residual_vt(n, V, T, dt=True, dv=True, dn=True)

# Displaying results
print("Residual Gibbs Energy: ", Gr)
print("Derivatives: ", derivatives)

Residual Gibbs Energy:  -713.3048693974467
Derivatives:  {'dt': 3.81552793271931, 'dv': -398.1289412715064, 'dn': array([-11.52100055, -51.51418507])}


### Residual Enthalpy $H^r(\vec{n}, V, T)$

In [12]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.enthalpy_residual_vt(n, V, T) # Calculate residual Enthalpy [bar L]

-1237.176213001435

You can obtain the next derivatives:

$$
\left(\frac{\partial H^{r}_{(\vec{n}, V, T)}}{\partial T}\right)_{V, \vec{n}} \quad
\left(\frac{\partial H^{r}_{(\vec{n}, V, T)}}{\partial V}\right)_{T, \vec{n}} \quad
\left(\frac{\partial H^{r}_{(\vec{n}, V, T)}}{\partial n_j}\right)_{V, T, n_{i \neq j}}
$$

Ask for derivatives as follows:

In [14]:
# Asking for all the derivatives
Hr, derivatives = model.enthalpy_residual_vt(n, V, T, dt=True, dv=True, dn=True)

# Displaying results
print("Residual Enthalpy: ", Hr)
print("Derivatives: ", derivatives)

Residual Enthalpy:  -1237.176213001435
Derivatives:  {'dt': 2.7262269081225208, 'dv': 222.65809494029838, 'dn': array([-130.30843347, -161.65842812])}


### Residual Entropy $S^r(n, V, T)$

In [15]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.entropy_residual_vt(n, V, T) # Calculate residual Entropy [bar L / K]

-1.746237812013294

You can obtain the next derivatives:

$$
\left(\frac{\partial S^{r}_{(\vec{n}, V, T)}}{\partial T}\right)_{V, \vec{n}} \quad
\left(\frac{\partial S^{r}_{(\vec{n}, V, T)}}{\partial V}\right)_{T, \vec{n}} \quad
\left(\frac{\partial S^{r}_{(\vec{n}, V, T)}}{\partial n_j}\right)_{V, T, n_{i \neq j}}
$$

Ask for derivatives as follows:

In [17]:
# Asking for all the derivatives
Sr, derivatives = model.entropy_residual_vt(n, V, T, dt=True, dv=True, dn=True)

# Displaying results
print("Residual Entropy: ", Sr)
print("Derivatives: ", derivatives)

Residual Entropy:  -1.746237812013294
Derivatives:  {'dt': 0.0021897892913883494, 'dv': 2.069290120706016, 'dn': array([-0.39595811, -0.36714748])}


### Residual heat Capacity at constant volume $C_V^r(n, V, T)$

In [18]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.cv_residual_vt(n, V, T) # Calculate residual heat capacity [bar L / K]

0.6569367874165049

### Residual Heat Capacity at constant pressure $C_P^r(n, V, T)$

In [19]:
n = [5.0, 5.0]  # number of moles [mol]
V = 1.0         # volume [L]
T = 300.0       # temperature [K]

model.cp_residual_vt(n, V, T) # Calculate residual heat capacity [bar L / K]

3.72361653132665

## Properties function of Pressure and Temperature $\left(P, T \right)$
We can obtain different bulk properties from a $A^r$ model specifying the
number of moles, pressure and temperature. But also, derivatives could be 
obtained if asked for.

All not asked derivatives will be set as `None`. The derivatives are returned
as a Python dictionary with the keys `dp`, `dt`, and `dn`, etc. respectively.
The compositional derivatives are always returned as `Numpy` arrays.

The following properties can be calculated:

### Volume

In [15]:
n = [5.0, 5.0]  # number of moles [mol]
P = 1.0         # pressure [bar]
T = 300.0       # temperature [K]

model.volume(n, P, T, root="stable")

245.62683294458913